In [1]:
import xarray as xr
import os
import matplotlib.pyplot as plt
%matplotlib inline
from mom6_design.postprocessing import combine_mom6_restarts, pad_ice_model, transpose_like_ref, fill_missing

In [2]:
# def combine_mom6_icebergs(file_name):
#     datasets = []
#     for nrange in [range(0,4), range(4,8), range(8,12), range(12,16)]:
#         flist = ['%s.%04i' %(file_name, n) for n in nrange]
#         print(flist)
#         datasets_sub = [xr.open_mfdataset(file) for file in flist]
#         ds_sub = xr.merge(datasets_sub)
#         print('###########debug###########')
#         print(ds_sub)
#         print('###########debug###########')
# #         ds_test = xr.open_mfdataset([os.path.join(path, file) for file in flist])
        
#         datasets.append(ds_sub)
# #     out = xr.concat(datasets, dim=concat_dim) 
#     out = xr.merge(datasets) # not sure why I have to do these seperately to work...
#     #This does concat existing dimensions, which is I guess what I want, but it would be nice if there would
#     out.attrs['jb_notes'] = 'Combined using xarray by Julius Busecke. See https://github.com/jbusecke/mom6_design for details'
#     return out

In [3]:
path = '/archive/Julius.Busecke/model_setups/xanadu_mom6_20181101/OM4p125_IAF_csf_JRA55do1-3_r5_cycle4/archive/restart/20020101'
path_iceberg = '/archive/Julius.Busecke/model_setups/xanadu_mom6_20181101/OM4p125_IAF_csf_JRA55do1-3_r5_cycle4/archive_alternative_with_icebergs/hide_from_fre/'
# odir = '/archive/Julius.Busecke/model_setups/xanadu_mom6_20181101/OM4p125_IAF_csf_JRA55do1-3_r5_cycle4/archive/restart_combined_experimental/new/'
odir = '/archive/Julius.Busecke/model_setups/xanadu_mom6_20181101/OM4p125_IAF_csf_JRA55do1-3_r5_cycle4/archive/restart_combined_final/'
refdir = '/archive/Julius.Busecke/model_setups/xanadu_mom6_20181101/OM4p125_IAF_csf_JRA55do1-3_r5_cycle4/archive/'
if not os.path.exists(odir):
    os.mkdir(odir)

In [4]:
# The icebergs are all empty?...JRAv1.3 does not include calving
ds_icebergs = xr.open_mfdataset('/archive/Julius.Busecke/model_setups/xanadu_mom6_20181101/OM4p125_IAF_csf_JRA55do1-3_r5_cycle4/archive_alternative_with_icebergs/hide_from_fre/icebergs.res.nc.*', drop_variables='i')
# print(ds_icebergs)

# just rename the first file...since they are all empty anyways...
file = '/archive/Julius.Busecke/model_setups/xanadu_mom6_20181101/OM4p125_IAF_csf_JRA55do1-3_r5_cycle4/archive_alternative_with_icebergs/hide_from_fre/icebergs.res.nc.0000'
!cp {file} {odir}/icebergs.res.nc

In [5]:
# ice model restarts
ds_icemodel = combine_mom6_restarts(os.path.join(path, 'ice_model.res.nc'))
# icemodel needs special patching (see `discover_debug_ice_model.ipynb`)
ds_icemodel = pad_ice_model(ds_icemodel)
print(ds_icemodel.nbytes/1e9, ds_icemodel)
ds_icemodel.to_netcdf(os.path.join(odir, 'ice_model.res.nc'))

# # # calving restart
ds_calving = combine_mom6_restarts(os.path.join(path, 'calving.res.nc'))
# calving just needs to be padded with zeros
ds_calving = fill_missing(ds_calving, 0)
print(ds_calving.nbytes/1e9, ds_calving)
ds_calving.to_netcdf(os.path.join(odir,'calving.res.nc'))

5.780316304 <xarray.Dataset>
Dimensions:          (Time: 1, aaxis_1: 1, aaxis_2: 4, xaxis_1: 2880, yaxis_1: 2240, zaxis_1: 6, zaxis_2: 5)
Coordinates:
  * yaxis_1          (yaxis_1) float64 1.0 2.0 3.0 ... 2.239e+03 2.24e+03
  * xaxis_1          (xaxis_1) float64 1.0 2.0 3.0 ... 2.879e+03 2.88e+03
  * zaxis_1          (zaxis_1) float64 1.0 2.0 3.0 4.0 5.0 6.0
  * zaxis_2          (zaxis_2) float64 1.0 2.0 3.0 4.0 5.0
  * aaxis_1          (aaxis_1) float64 1.0
  * aaxis_2          (aaxis_2) float64 1.0 2.0 3.0 4.0
  * Time             (Time) float64 1.0
Data variables:
    flux_u           (Time, yaxis_1, xaxis_1) float64 dask.array<shape=(1, 2240, 2880), chunksize=(1, 2240, 2880)>
    flux_v           (Time, yaxis_1, xaxis_1) float64 dask.array<shape=(1, 2240, 2880), chunksize=(1, 2240, 2880)>
    flux_t           (Time, yaxis_1, xaxis_1) float64 dask.array<shape=(1, 2240, 2880), chunksize=(1, 2240, 2880)>
    flux_q           (Time, yaxis_1, xaxis_1) float64 dask.array<shape=(1, 2240,

## Combining the MOM restart files
This takes a long time, but from my tests this did not need any padding? (Or did I forget some step?)

In [ ]:
# MOM restarts (these need a slight bit more TLC)
for filename in ['MOM.res.nc']+['MOM.res_%i.nc' %i for i in range(1,17)]:
    print(filename)
    ds_mom = combine_mom6_restarts(os.path.join(path, filename))
    print(ds_mom)
    
    #this allows datasets with different x dimensions to be logically merged? 
    print('Filesize: %s GB' %str(ds_mom.nbytes/1e9))
    if ds_mom.nbytes/1e9 > 5.0:
        print('Something is kaput')
        break
        
    print('Writing File')
    ds_mom.to_netcdf(os.path.join(odir, filename))
    del ds_mom

MOM.res.nc
<xarray.Dataset>
Dimensions:  (Layer: 75, Time: 1, lath: 2240, lonh: 2880)
Coordinates:
  * lath     (lath) float64 -83.62 -83.58 -83.54 -83.5 ... 89.86 89.92 89.97
  * lonh     (lonh) float64 -298.6 -298.5 -298.4 -298.3 ... 61.05 61.16 61.28
  * Layer    (Layer) float64 1.0 3.0 5.0 7.0 ... 3.338e+03 3.591e+03 5.111e+03
  * Time     (Time) timedelta64[ns] 90353 days 01:16:18.871345
Data variables:
    Temp     (Time, Layer, lath, lonh) float64 dask.array<shape=(1, 75, 2240, 2880), chunksize=(1, 75, 2240, 2880)>
Attributes:
    jb_notes:  Combined using xarray by Julius Busecke. See https://github.co...
Filesize: 3.870761568 GB
Writing File
MOM.res_1.nc
<xarray.Dataset>
Dimensions:  (Layer: 75, Time: 1, lath: 2240, lonh: 2880)
Coordinates:
  * lath     (lath) float64 -83.62 -83.58 -83.54 -83.5 ... 89.86 89.92 89.97
  * lonh     (lonh) float64 -298.6 -298.5 -298.4 -298.3 ... 61.05 61.16 61.28
  * Layer    (Layer) float64 1.0 3.0 5.0 7.0 ... 3.338e+03 3.591e+03 5.111e+03
  * Ti

Lets see if those restarts work right out of the box with my discover setup... otherwise use [FRE-nctools](https://github.com/NOAA-GFDL/FRE-NCtools) as suggeste [here](https://github.com/NOAA-GFDL/MOM6/issues/958).

## QC the output files
I am having trouble in particular with the ice...model crashed due to icebergs being born on land.
This is due to very high values on land!


In [ ]:
ds_check_calving = xr.open_mfdataset(os.path.join(odir,'calving.res.nc'))
ds_check_calving.stored_ice.where(ds_check_calving.stored_ice>0).plot(col='zaxis_1', col_wrap=3, robust=True)

plt.figure()
ds_check_calving.stored_heat.plot(col='zaxis_1', col_wrap=3, robust=True)

plt.figure()
ds_check_calving.iceberg_counter_grd.plot(robust=True)

In [ ]:
ds_check_icemodel = xr.open_mfdataset(os.path.join(odir,'ice_model.res.nc'))
ds_check_icemodel

In [ ]:
ds_check_icemodel.runoff_hflx.plot()

# mask out all necessary files

This is old. Delete once the refactored merging above is successful

In [ ]:
# load ocean mask...
griddir = '/archive/Julius.Busecke/model_setups/xanadu_mom6_20181101/OM4p125_IAF_csf_JRA55do1-3_r5_cycle4/input_files/grid/untarred/'
mask = xr.open_dataset(os.path.join(griddir, 'ocean_mask.nc'))
print(mask)
mask.mask.plot()

In [ ]:
# # convert dimensions of mask to calving file
# mask_ocean = mask.mask.rename({'nx':'xaxis_1', 'ny':'yaxis_1'})
# ds_check_calving.stored_ice.isel(zaxis_1=0).where(mask_ocean==0).plot()

In [ ]:
ds_calving_padded = ds_check_calving.where(mask_ocean).fillna(0)
ds_calving_padded.stored_ice.isel(zaxis_1=0).where(mask_ocean==0).plot()

In [ ]:
ds_icemodel_padded = ds_check_icemodel.where(mask_ocean).fillna(0)
# ds_icemodel_padded.calving_hflx.where(mask_ocean==0).plot()
# ds_icemodel_padded

In [ ]:
ds_icemodel_padded.to_netcdf(os.path.join(odir, 'ice_model.res.nc.padded'))

In [ ]:
ds_calving_padded.to_netcdf(os.path.join(odir, 'calving.res.nc.padded'))

In [ ]:
test = xr.open_dataset(os.path.join(odir, 'ice_model.res.nc.padded'))#.where(mask_ocean==0)
# test.coszen.plot(robust=True)
test

In [ ]:
test.h_ice.plot(robust=True, col='zaxis_2', vmax=1)

In [ ]:
test.part_size.sum('zaxis_1').plot(vmax=1)

In [ ]:
test.part_size.plot(col='zaxis_1', vmin=1, vmax=1.01)

In [ ]:
# Check output visually against the 1 sec output of the rest run
